# NYC 311 Complaints 2017

For more information on this dataset, see the following articles:
* The _Wired_ magazine article ["What a Hundred Million Calls to 311 Reveal About New York."](https://www.wired.com/2010/11/ff_311_new_york/).
* Ariel White and Kris-Stella Trump's research paper ["The Promises and Pitfalls of 311 Data"](https://arwhite.mit.edu/sites/default/files/images/White%20Trump%20-%20Promises%20Pitfalls%20311%20Data%20-%20UAR%202017.pdf) in [_Urban Affairs Review_](https://journals.sagepub.com/doi/abs/10.1177/1078087416673202).

I used the White and Trump paper, in paritcular, when the making data-cleaning assumptions (e.g., what to clean vs. what to remove) for this project.

In [ ]:
import os
from time import time
import requests
from bs4 import BeautifulSoup
from requests import HTTPError
import numpy as np
import pandas as pd
from sodapy import Socrata

## Data Sourcing

The following data sources were used:

* [NYC 311 Complaints](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)
* [2010 Census Population by ZIP Code](https://blog.splitwise.com/2013/09/18/the-2010-us-census-population-by-zip-code-totally-free/)
* [NYC Neighborhod ZIP Codes](https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm)

Inspection of the data revealed non-NYC cities and ZIP codes. Thus, as the focus of this project is on NYC-related complaints, complaints associated with non-NYC locations are removed from the dataset.

### 311 Complaints

In [ ]:
!pip install pyarrow
complaints = pd.read_parquet('data/nyc-311-complaints.parquet.gzip')

#### Progamatically Sourcing the Data

The [`sodapy`](https://github.com/xmunoz/sodapy) package used to interface with the [Socrata Open Data (SODA) API](https://dev.socrata.com/) and return the raw 311 complaints data in JSON format. Let's define two (perhaps unnecessary) utility functions, one to format the fields to use in the SoQL query, and another to safely pull the results.


In [ ]:
def format_fields(*fields):
    """ Formats arbitrary nubmer of fields for the SoQL query """
    return ','.join(fields)


def get_complaints_data(*fields, year=2017, limit=20000000):
    """ Pull certain `fields` for a given `year` from NYC Open Data's
    SODA API """
    try:
        with Socrata('data.cityofnewyork.us', None) as client:
            results = client.get(
                'fhrw-4uyv',
                content_type='json',
                select=format_fields(*fields),
                where=f"created_date between '{year}-01-01T00:00:00' and '{year}-12-31T23:59:59'",
                limit=limit
            )
        return results
    except HTTPError as e:
        print(e)

Now let's use these functions to get our hands on the 311 data for 2017! Note that we're not making a request with an app token, so we'll get a warning that we'll safely ignore for the purposes of this exercise.

In [ ]:
r = get_complaints_data(
    'unique_key',
    'created_date',
    'complaint_type',
    'descriptor',
    'borough',
    'city',
    'incident_zip',
    'incident_address',
    'latitude',
    'longitude'
)

In [ ]:
complaints = pd.DataFrame.from_records(r)

Converting to datetime takes awhile, so a TODO is to find a faster way to do this.

In [ ]:
complaints.sample(10)

Note that city is _often_ neighborhood, but borough is also logged here as well.

In [ ]:
complaints = complaints.replace('Unspecified', np.nan)

In [ ]:
complaints[complaints['borough'].isnull()].sample(10)

Let's just add a small sanity check to make sure we only have data from 2017...

In [ ]:
assert complaints.created_date.min().year == 2017
assert complaints.created_date.max().year == 2017

#### Investigating Missing Values

First, let's look at the percentage of missing values in each column.

In [ ]:
complaints.apply(lambda x: round(100 * x.isnull().mean(), 2))

In [ ]:
complaints.borough.value_counts()

In [ ]:
complaints.query("borough == 'Unspecified'").isnull().mean()

Given that we have unspecified boroughs and our analysis is borough-based, for the present analysis we can simply replace the non-borough cities with missing values and fill in any null borough values with any bourgh values logged in the `city` field, and then remove the now redundant `city` field.

In [ ]:
complaints['city'].isin(['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'])
complaints.loc[is_borough, 'city'] = np.nan
complaints.loc[:, 'borough'] = (
    complaints.loc[:, 'borough']
              .combine_first(complaints['city'])
)
complaints.drop('city', axis=1, inplace=True)
complaints.sample(10)

### Population Data by ZIP Code

Note that this dataset technically contains [ZCTA (ZIP Code Tabulation Area)](https://www.census.gov/geo/reference/zctas.html) codes, not ZIP codes. However, they are treated as equivalent for the purpose of this exercise.

In [ ]:
def get_population_by_zip(url):
    """ Gets 2010 Census population by ZIP code data """
    try:
        population_by_zip = pd.read_csv(url)
        return population_by_zip
    except HTTPError as e:
        print(e)

In [ ]:
population_url = ('https://s3.amazonaws.com/SplitwiseBlogJB/'
                         '2010+Census+Population+By+Zipcode+(ZCTA).csv')

population_by_zip = get_population_by_zip(population_url)

In [ ]:
population_by_zip.columns = 'zip_code population'.split(' ')
population_by_zip.head(10)

### NYC ZIP Code Data

As mentioned above, brief inspection of the 311 complaints dataset revealed cities that are not in NYC. Thus, as part of my preprocessing I filtered out these observations.

Data on the ZIP codes associated with NYC neighborhoods comes from the New York State Department of Health. The HTML table this data is stored in is not in the most convenient format for data analysis, so the data must be scraped and turned into ["tidy"](https://r4ds.had.co.nz/tidy-data.html) format (i.e., basically a statistician's analogue of Codd's [3NF](https://en.wikipedia.org/wiki/Third_normal_form) familiar to data engineers).

First, let's create two utility functions, the first to scrape the table from the Department of Health website, and the second to "tidy" the data into a minimal table of borough by ZIP code data.

In [ ]:
def scrape_nyc_zips(url):
    """ Scrapes table of NYC zipcodes from NYC Department of Health """
    try:
        r = requests.get(url)
        return r
    except HTTPError as e:
        print("NYC neighborhood ZIP code lookup table not found:", e)


# TODO: Refactor to have utility functions for, for example, the
# "tidying" aspects and the conversion aspects ... and rename this
# function to something more sensible
def tidy_nyc_zips(html):
    """ Wrangle HTML table of NYC ZIP codes into a "tidy" data frame

    Args:
        html (requests.models.Response):

    Returns:
        pandas.DataFrame:
    """

    # TODO: This seems too ugly and hacky so find a more elegant
    # solution
    borough_zips = (
        pd.read_html(html.content, header=0)[0]
          .reset_index()
    )

    borough_zips.loc[borough_zips['ZIP Codes'].isnull(), 'Borough'] = np.nan
    borough_zips.loc[:, 'ZIP Codes'] = \
        borough_zips.loc[:, 'ZIP Codes'].str.replace(' ', '')

    borough_zips.loc[:, 'ZIP Codes'] = (
        borough_zips.loc[:, 'ZIP Codes']
                    .combine_first(borough_zips['Neighborhood'])
    )

    # TODO: keep the neighborhood information, even though it's not
    # currently necessary for this analysis
    borough_zips.drop('Neighborhood', axis=1, inplace=True)
    borough_zips.loc[:, 'Borough'] = \
        borough_zips.loc[:, 'Borough'].ffill()

    # Overwrite the comma-separated string "list" in the cell
    # with an actual list of integers
    borough_zips.loc[:, 'ZIP Codes'] = (
        borough_zips.loc[:, 'ZIP Codes']
                    .apply(lambda x: x.split(','))
    )

    # TODO: Write utility function for this pattern
    borough_zips = (
        borough_zips.set_index(['index', 'Borough'])
                    .loc[:, 'ZIP Codes']
                    .apply(pd.Series) # Expand the list of 
                    .stack()
                    .reset_index()
    )

    borough_zips.drop(['index', 'level_2'], axis=1, inplace=True)
    borough_zips.columns = \
        'borough zip_code'.split(' ')
    borough_zips.loc[:, 'zip_code'] = \
        borough_zips.loc[:, 'zip_code'].astype(int)

    return borough_zips

In [ ]:
nyc_zips_url = ('https://www.health.ny.gov/statistics/cancer/registry/appendix/'
                'neighborhoods.htm')

html = scrape_nyc_zips(nyc_zips_url)
nyc_zips = tidy_nyc_zips(html)

nyc_zips.head(10)

In [ ]:
population_by_zip_nyc = nyc_zips.merge(
    population_by_zip,
    on='zip_code',
    how='left'
)

In [ ]:
population_by_zip_nyc.head(10)

In [ ]:
population_by_zip_nyc[population_by_zip_nyc['population'].isnull()]

Note that ZIP code 11695 is (according to Google) Far Rockaway.

## Data Cleaning

Given the size of the dataset (viz., about 19.5 million observations), string values are converted to `pandas` [categorical](https://pandas.pydata.org/pandas-docs/stable/categorical.html) variables, which are internally stored as integers and thus cut down on memory usage when slicing and dicing the data frame.

### Data Type Conversion

The main idea here is to cut down on the number of variables stored as text in order to decrease the memory used by `pandas`, which at the start is as follows:

In [ ]:
complaints.memory_usage()

In [ ]:
complaints.loc[:, 'unique_key'] = \
    complaints.loc[:, 'unique_key'].astype(int)

complaints.loc[complaints['borough'].eq('Unspecified'), 'borough'] = None

complaints.loc[:, 'borough'] = \
    complaints.loc[:, 'borough'].astype('category')

complaints.loc[:, 'complaint_type'] = \
    complaints.loc[:, 'complaint_type'].astype('category')

complaints.loc[:, 'created_date'] = \
    complaints.loc[:, 'created_date'].apply(pd.to_datetime)

In [ ]:
complaints.memory_usage()
complaints.dtypes

In [ ]:
complaints.loc[:, 'incident_zip'] = \
    complaints.loc[:, 'incident_zip'].apply(pd.to_numeric, errors='coerce')

### Imputing Values for "Unspecified" Boroughs

Brief exploration of the 311 complaints dataset revels that borough is missing for many incidents associated with Queens. For example, a value of `Unspecified` shows up for Forest Hills, Hollis, and other neighborhoods in Queens.

In [ ]:
is_unspecified = complaints['borough'].eq('Unspecified')
complaints.loc[is_unspecified, 'borough'] = np.nan

In [ ]:
complaints[complaints['borough'].isnull() & complaints['city'].isnull()].sample(10)
complaints[complaints['complaint_type'].str.contains('request', case=False)].complaint_type.value_counts()
is_request = complaints['complaint_type'].str.contains('request', case=False)
is_benefit = complaints['complaint_type'].eq('Benefit Card Replacement')
no_requests = complaints[~is_request & ~is_benefit].copy()
# Benefit Card Replacement
# DCA / DOH New License Application Request
# DOF Parking - Payment Issue
# School Maintenance
# Literature Request
# Street Light Condition
# Forms
# Illegal Parking
# DOF Parking - DMV Clearance
# DCA / DOH New License Application Request

In [ ]:
no_requests[no_requests['borough'].isnull() & complaints['city'].isnull()].sample(10)

In [ ]:
complaints['city'].value_counts()
complaints[complaints['borough'].isnull() & complaints['city'].isnull()].park_borough.value_counts()
complaints[complaints['complaint_type'].str.contains('request', case=False)].complaint_type.value_counts()

### (Minimal) Text Standardization and Cleaning

`ALL CAPS` values were changed to `Title Case` for the `borough` and `city` fields.

No further cleanup or standardization was attempted `complaint_type` or `descriptor` fields, as the capitalizations and conventions here (e.g., acronmyms) here often appear meaningful.

In [ ]:
complaints.loc[:, 'borough'] = complaints.loc[:, 'borough'].str.title()
complaints.loc[:, 'city'] = complaints.loc[:, 'city'].str.title()
complaints.sample(15)

In [ ]:
complaints.isnull().mean()

Now let's get the last of the "Unspecified" boroughs that we can (given the minimal cleanup and data sourcing that we've done).

In [ ]:
nrow_start = complaints.shape[0]
complaints = complaints.merge(
    population_by_zip_nyc,
    left_on='incident_zip',
    right_on='zip_code',
    how='left'
)
assert complaints.shape[0] == nrow_start

In [ ]:
complaints.loc[:, 'borough'] = (
    complaints.loc[:, 'borough_x']
              .combine_first(complaints['borough_y'])
)
complaints.drop(['borough_x', 'borough_y'], axis=1, inplace=True)

In [ ]:
complaints.isnull().mean()

In [ ]:
is_borough = complaints['city'].isin(['Brooklyn', 'Queens', 'Staten Island', 'Manhattan', 'Bronx'])
complaints.loc[~is_borough, 'complaints'] = np.nan

complaints.loc[:, 'borough'] = (
    complaints.loc[:, 'borough']
              .combine_first(complaints['city'])
)

complaints.loc[:, 'borough'] = (
    complaints.loc[:, 'borough_x']
              .combine_first(complaints['borough_y'])
)

### Removing Non-NYC Observations

Given that the questions in this exercise deal with NYC, any complaint associated with a ZIP code outside of the city's boundaries is removed.

<img src="geocode.png" alt="Drawing" style="width: 400px;"/>

In [ ]:
is_nyc = complaints['zipcode'].isin(nyc_zips['zipcode'])
complaints = complaints.loc[is_nyc, :]

There's much more I'd like to do to clean up the dataset and impute missing values ... but you've got to stop somewhere.

### Wrapping Up

The cleaned version of the dataset is then saved for later analysis using the columnar [Apache Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) format, which is significantly faster to read than the corresponding CSV.

In [ ]:
complaints.to_parquet('data/nyc-311-complaints.parquet.gzip')